In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('nyc-rolling-sales.csv', encoding='utf-8')
df_old_size = df.shape[0]

In [ ]:
df.isnull().any()

In [ ]:
df.info()

In [ ]:
df['NEIGHBORHOOD'].unique()

In [ ]:
df = df[df['NEIGHBORHOOD'] != ' ']
df['NEIGHBORHOOD'] = pd.factorize(df['NEIGHBORHOOD'])[0]

In [ ]:
df['BUILDING CLASS CATEGORY'].unique()

In [ ]:
df = df[df['BUILDING CLASS CATEGORY'] != ' ']
df['BUILDING CLASS CATEGORY'] = pd.factorize(df['BUILDING CLASS CATEGORY'])[0]

In [ ]:
df['TAX CLASS AT PRESENT'].unique()

In [ ]:
df = df[df['TAX CLASS AT PRESENT'] != ' ']
#df['TAX CLASS AT PRESENT'] = pd.factorize(df['TAX CLASS AT PRESENT'])[0]

def simplify(f):
    if '1' in f:
        return 1
    if '2' in f:
        return 2
    if '3' in f:
        return 3
    if '4' in f:
        return 4

df['TAX CLASS AT PRESENT'] = df['TAX CLASS AT PRESENT'].apply(simplify)

In [ ]:
df['BUILDING CLASS AT PRESENT'].unique()

In [ ]:
df = df[df['BUILDING CLASS AT PRESENT'] != ' ']
df['BUILDING CLASS AT PRESENT'] = pd.factorize(df['BUILDING CLASS AT PRESENT'])[0]

In [ ]:
df['EASE-MENT'].unique()

In [ ]:
del df["EASE-MENT"]

In [ ]:
df['ADDRESS'].unique()

In [ ]:
def clear(s):
    ns = s.split(' ', 1)
    l = len(ns)
    return ns[l - 1]

df['ADDRESS'] = df['ADDRESS'].apply(clear)

In [ ]:
df = df[df['ADDRESS'] != ' ']
df['ADDRESS'] = pd.factorize(df['ADDRESS'])[0]

In [ ]:
df['APARTMENT NUMBER'].unique()

In [ ]:
del df['APARTMENT NUMBER']

In [ ]:
df['LAND SQUARE FEET'].unique()

In [ ]:
col = pd.to_numeric(df['LAND SQUARE FEET'], errors='coerce')
allCs = col.size
nullCs = col[col.isnull()].size
print('count', allCs, sep=': ') 
print('nulls count', nullCs, sep=': ')
print('percent of nulls', nullCs/allCs*100, sep=': ')

In [ ]:
df['LAND SQUARE FEET'] = pd.factorize(df['LAND SQUARE FEET'])[0]

In [ ]:
df['GROSS SQUARE FEET'].unique()

In [ ]:
col = pd.to_numeric(df['GROSS SQUARE FEET'], errors='coerce')
allCs = col.size
nullCs = col[col.isnull()].size
print('count', allCs, sep=': ') 
print('nulls count', nullCs, sep=': ')
print('percent of nulls', nullCs/allCs*100, sep=': ')

In [ ]:
df['GROSS SQUARE FEET'] = pd.factorize(df['GROSS SQUARE FEET'])[0]


In [ ]:
df['BUILDING CLASS AT TIME OF SALE'].unique()

In [ ]:
df['BUILDING CLASS AT TIME OF SALE'] = pd.factorize(df['BUILDING CLASS AT TIME OF SALE'])[0]

In [ ]:
df['SALE PRICE'].unique()

In [ ]:
col = pd.to_numeric(df['SALE PRICE'], errors='coerce')
allCs = col.size
nullCs = col[col.isnull()].size
print('count', allCs, sep=': ') 
print('nulls count', nullCs, sep=': ')
print('percent of nulls', nullCs/allCs*100, sep=': ')

In [ ]:
df['SALE PRICE'] = col
df = df[pd.notnull(df['SALE PRICE'])]
df = df[df['SALE PRICE'] != 0]

In [ ]:
df['SALE PRICE'].unique()

In [ ]:
df.hist(['SALE PRICE'])

In [ ]:
df = df[df['SALE PRICE'] < 1.8 * 10**8]
df.hist(['SALE PRICE'])

In [ ]:
df['SALE DATE'].unique()

In [ ]:
"""
col = pd.to_datetime(df['SALE DATE'], errors='coerce')
allCs = col.size
nullCs = col[col.isnull()].size
print('count', allCs, sep=': ') 
print('nulls count', nullCs, sep=': ')
print('percent of nulls', nullCs/allCs*100, sep=': ')
"""

In [ ]:
"""
from datetime import date
from datetime import datetime
def simplify(f):
    return (date.today() - datetime.strptime(f, "%Y-%m-%d").date()).days

df['SALE DATE'] = col.astype('string').apply(simplify)
df = df[pd.notnull(df['SALE DATE'])]
"""
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'], errors='coerce')
df = df[df['SALE DATE'] != None]
df['SALE DATE'] = pd.to_numeric(df['SALE DATE'], errors='coerce')
#del df['SALE DATE']

In [ ]:
df_size = df.shape[0]
print('rows removed', df_old_size - df_size, sep=': ')
print('rows now', df_size, sep=': ')

In [ ]:
df.info()

In [ ]:
x = df.loc[:, df.columns != 'SALE PRICE']
y = df['SALE PRICE']
train_points, test_points, train_values, test_values = train_test_split(x, y, random_state=104, test_size=0.25, shuffle=False, stratify=None)

In [ ]:
rf_model = ensemble.RandomForestRegressor(n_estimators=100)
rf_model.fit(train_points, train_values)
test_predict_rf = rf_model.predict(test_points)
print(mean_absolute_error(test_values, test_predict_rf))

In [ ]:
x = df.loc[:, df.columns != 'SALE PRICE']
y = df['SALE PRICE']
train_points, test_points, train_values, test_values = train_test_split(x, y, random_state=104, test_size=0.23, shuffle=False, stratify=None)

In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 13)
xg_reg.fit(train_points, train_values)
xgb_predict = xg_reg.predict(test_points)
print(mean_absolute_error(test_values, xgb_predict))